In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('dataincode') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

24/10/24 00:06:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Importar funções 
from IPython.display import display, HTML

In [ ]:
%run ./Includes/Utils.ipynb

In [ ]:
%run ./Includes/Datasets.ipynb

## Criar Dataframes e escrever tabelas no catalogo

In [ ]:
# Inicio rapido

# help(create_dataframe)
# print(lista_amostras)

In [ ]:
init_data_df = create_dataframe(columns_schema, init_data)

In [ ]:
# init_data_df.show(5)

### Criar Tabela
- ``` df.writeTo(t).create() ``` é equivalente a ``` CREATE TABLE AS SELECT ```
- ``` df.writeTo(t).replace() ``` é equivalente a ``` REPLACE TABLE AS SELECT ```
- ``` df.writeTo(t).append() ``` é equivalente a ``` INSERT INTO ```
- ``` df.writeTo(t).overwritePartitions() ``` é equivalente a dynamic ``` INSERT OVERWRITE ```
fonte: https://iceberg.apache.org/docs/1.6.0/spark-writes/?h=df.writeto%28t%29.create%28%29#writing-with-dataframes

In [ ]:
# Escrever tabela no storage
(
    init_data_df
    .writeTo("iceberg.bronze.vendas")
    .partitionedBy("country")
    .createOrReplace()
)

In [ ]:
## Visualizar os dados simples e leve

spark.sql("SELECT * FROM iceberg.bronze.vendas").show()

In [ ]:
## Visualizar os dados

spark.sql("SELECT * FROM iceberg.bronze.vendas").toPandas()

In [ ]:
## Inserir novos dados

feb_data_df = create_dataframe(columns_schema, feb_data)

feb_data_df.writeTo("iceberg.bronze.vendas").append()

### Primeiras Impressões

In [ ]:
## Descrever tabela

spark.sql("DESCRIBE iceberg.bronze.vendas")

In [ ]:
## Inspecionar tabelas

spark.sql("SELECT * FROM iceberg.bronze.vendas.history").show()

In [ ]:
spark.sql("SHOW TBLPROPERTIES iceberg.bronze.vendas").show()

In [ ]:
spark.sql("SELECT COUNT(*) FROM iceberg.bronze.vendas").show()